In [ ]:
!pip install transformers

     |████████████████████████████████| 757kB 2.8MB/s 
     |████████████████████████████████| 890kB 12.5MB/s 
     |████████████████████████████████| 3.0MB 16.8MB/s 
     |████████████████████████████████| 1.1MB 41.7MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.43-cp36-none-any.whl size=893260 sha256=9d3e6572433efb8d5e18b72646b993a36eb9172130ab29be54c5aa7e5eb8720b
  Stored in directory: /root/.cache/pip/wheels/29/3c/fd/7ce5c3f0666dab31a50123635e6fb5e19ceb42ce38d4e58f45
Successfully built sacremoses


In [ ]:
import torch
import numpy as np
from transformers import GPT2Tokenizer, GPT2LMHeadModel
from transformers import AdamW, get_linear_schedule_with_warmup
from torch.utils.data import Dataset
from torch.utils.data import Dataset, DataLoader
import os

In [ ]:
device = 'cpu'
if torch.cuda.is_available():
    device = 'cuda'
    
tokenizer = GPT2Tokenizer.from_pretrained('gpt2')
model = GPT2LMHeadModel.from_pretrained('gpt2')
model = model.to(device)

Some weights of GPT2LMHeadModel were not initialized from the model checkpoint at gpt2 and are newly initialized: ['h.0.attn.masked_bias', 'h.1.attn.masked_bias', 'h.2.attn.masked_bias', 'h.3.attn.masked_bias', 'h.4.attn.masked_bias', 'h.5.attn.masked_bias', 'h.6.attn.masked_bias', 'h.7.attn.masked_bias', 'h.8.attn.masked_bias', 'h.9.attn.masked_bias', 'h.10.attn.masked_bias', 'h.11.attn.masked_bias', 'lm_head.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
tokenizer.max_len

1024

In [ ]:
d = tokenizer.encode("the fabric of the universe", return_tensors='pt')

In [ ]:
d

tensor([[1169, 9664,  286,  262, 6881]])

In [ ]:
tokenizer.decode(model.generate(d.cuda(), do_sample=True, 
    max_length=100, 
    top_k=50)[0],skip_special_tokens=True)

Setting `pad_token_id` to 50256 (first `eos_token_id`) to generate sequence


'the fabric of the universe - not the human being that we believe exists in this world). For these reasons, God is one thing, but he is also the reason that our present world is so radically different from the one that we hold so dear. As Einstein himself commented, "When we get a bit too much faith in God, I\'m convinced there really is nothing more fundamental to man\'s nature." (Ibid.)\n\nGod and Time\n\nThe fact of such great difference in world'

In [ ]:
model.lm_head

Linear(in_features=768, out_features=50257, bias=False)

In [ ]:
class Data(Dataset):

  def __init__(self, tokenizer, df, block_size=512):

    block_size = block_size - (
        tokenizer.max_len - tokenizer.max_len_single_sentence
    )

    tokenized_text = tokenizer.convert_tokens_to_ids(tokenizer.tokenize(df))

    self.examples = []
    for i in range(0, len(tokenized_text) - block_size +1, block_size):

      self.examples.append(
          tokenized_text[i : i + block_size] + [tokenizer.eos_token_id]
      )

  def __len__(self):
    return len(self.examples)

  def __getitem__(self, item):
    return torch.tensor(self.examples[item], dtype=torch.long)

In [ ]:
import pandas as pd

In [ ]:
data = pd.read_csv("drive/My Drive/machine_tran/data1.csv")["en"].to_list()

In [ ]:
len(data)

20073

In [ ]:
valid = Data(tokenizer," ".join(data[:800]))
train = Data(tokenizer," ".join(data[800:]))

In [ ]:
batch_size = 1

train_loader= DataLoader(train,shuffle=False,batch_size=batch_size,)
valid_loader =DataLoader(valid,shuffle=False,batch_size=batch_size,)

In [ ]:
epochs = 10
learning_rate = 0.00002
warmup_steps = 2000

In [ ]:

optimizer = AdamW(model.parameters(), lr=learning_rate)
scheduler = get_linear_schedule_with_warmup(optimizer, num_warmup_steps=warmup_steps, num_training_steps=-1)


In [ ]:
d = next(iter(train_loader))

In [ ]:
d.shape

torch.Size([2, 513])

In [ ]:
model.train()
for epoch in range(EPOCHS):

  print(f"Epoch {epoch} started")

  sum_loss = 0
  val_loss = 0
  model.train()
  for data in train_loader:

    output = model(data.cuda(), labels= data.cuda())


    loss, logits = output[:2]
    loss.backward()

    sum_loss += loss.item()
    optimizer.step()
    scheduler.step()
    optimizer.zero_grad()


  # model.eval()
  # for data in valid_loader:

  #   output = model(data.cuda(), labels= data.cuda())


  #   loss, logits = output[:2]

  #   val_loss += loss.item()

  print(f"training loss { sum_loss / len(train_loader)}")



Epoch 0 started


/pytorch/torch/csrc/utils/python_arg_parser.cpp:756: UserWarning: This overload of add_ is deprecated:
	add_(Number alpha, Tensor other)
Consider using one of the following signatures instead:
	add_(Tensor other, *, Number alpha)


training loss 3.573071580515443
Epoch 1 started
training loss 3.2052259990090977
Epoch 2 started
training loss 3.120949829765672
Epoch 3 started
training loss 3.1209272521950626
Epoch 4 started
training loss 3.121183654262681
Epoch 5 started
training loss 3.1209100142563924
Epoch 6 started
training loss 3.120586305955062
Epoch 7 started
training loss 3.119599896098914
Epoch 8 started
training loss 3.1208393678019934
Epoch 9 started
training loss 3.121278724064528


In [ ]:
f = tokenizer.encode("please", return_tensors='pt')

In [ ]:
tokenizer.decode(model.generate(f.cuda(), do_sample=True, 
    max_length=100, 
    top_k=50)[0],skip_special_tokens=True)

Setting `pad_token_id` to 50256 (first `eos_token_id`) to generate sequence


'please and let us do it. He will bring about a lasting and lasting kindness for all the people that will be around him.” They will have to go on an upright walk for the rest of their lives, so that you may have peace. Look! The sun will shine out like a cloud, and with it will be a full moon like those who were alive at its light. Then they will walk straightly in their paths, not looking down on the sun, as though they were'